In [8]:
import numpy as np
from numpy import random as rand

In [9]:
def create_matrix(n_rows, n_columns):
    scores_array = rand.uniform(1,10,size=(n_rows, n_columns))
    return scores_array
 

In [10]:
# Test function
scores = create_matrix(20,20)

print(scores)

[[2.22615164 7.53709899 7.78273411 7.34903367 3.29736239 7.41828996
  8.35119156 4.27552607 3.50035782 6.92560144 8.46641421 6.16583816
  6.600214   3.69939627 6.0065867  4.02042111 3.36822142 4.45139341
  6.18699369 5.85508094]
 [7.08541402 4.11659285 7.5890232  5.23041967 3.17502895 2.70027791
  4.48312539 4.03230916 2.51934835 3.85467839 5.27500607 4.54225115
  9.75007707 4.66912968 5.82691654 9.22066443 7.72232185 9.14606674
  7.40466476 2.24322699]
 [7.13736039 1.96695585 8.35158221 6.00324855 7.61911644 8.19603295
  2.96659273 5.47936467 7.82452262 6.67251212 7.99761687 5.0247863
  3.61566418 6.01040826 4.87169365 8.43111985 9.90579311 6.01025466
  6.82723147 5.87092732]
 [7.18595301 4.72123424 6.06063371 4.77548687 3.9000328  2.02190459
  5.06362169 4.47546169 8.77037761 1.55368574 6.0866687  2.25851066
  8.52126774 2.23106941 5.80068666 8.52260433 1.25345226 2.48149402
  3.81838263 4.99792876]
 [8.92128995 8.80768215 6.73764081 7.9298429  6.45603124 3.46856567
  7.27895348 5.06

In [11]:
def calculate_scores_for_prediction(scores_array, percent):
    scores_for_prediction = scores_array.copy()
    number_of_zeros = int(np.round((1-percent)*scores_array.shape[0]*scores_array.shape[1]))
    choices = np.random.choice(scores_array.size, number_of_zeros, replace=False)
    scores_for_prediction.ravel()[choices] = 0 
    
    return scores_for_prediction

In [12]:
# Test function
scores_predict = calculate_scores_for_prediction(scores, 0.7)

print(scores_predict)

[[2.22615164 7.53709899 7.78273411 7.34903367 3.29736239 7.41828996
  0.         0.         3.50035782 6.92560144 8.46641421 6.16583816
  0.         0.         6.0065867  4.02042111 3.36822142 4.45139341
  6.18699369 0.        ]
 [0.         4.11659285 7.5890232  0.         3.17502895 0.
  0.         4.03230916 2.51934835 0.         0.         4.54225115
  0.         0.         5.82691654 0.         7.72232185 9.14606674
  7.40466476 0.        ]
 [7.13736039 1.96695585 8.35158221 0.         7.61911644 8.19603295
  0.         0.         7.82452262 0.         7.99761687 5.0247863
  3.61566418 0.         0.         8.43111985 0.         6.01025466
  6.82723147 5.87092732]
 [7.18595301 4.72123424 6.06063371 0.         3.9000328  0.
  5.06362169 0.         8.77037761 1.55368574 6.0866687  2.25851066
  0.         2.23106941 5.80068666 8.52260433 1.25345226 2.48149402
  3.81838263 4.99792876]
 [8.92128995 8.80768215 6.73764081 7.9298429  6.45603124 3.46856567
  7.27895348 0.         9.2570884

In [13]:
def Jaccard_Similarity(x, y):
    intersection = len(set(x) & set(y))
    union = len(set(x) | set(y))
    return intersection/float(union)

In [14]:
# Test function
Jaccard_Similarity(scores_predict[:,0], scores_predict[:,4])

0.03225806451612903

In [15]:
def Dice_Similarity(x,y):
    intersection = len(set(x) & set(y))
    return (2*intersection)/(len(x) + len(y))

In [16]:
# Test function
Dice_Similarity(scores_predict[:,0], scores_predict[:,4])

0.05

In [17]:
def Cosine_Similarity(x,y):
    dot_product = np.dot(x,y)
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y)
    return dot_product/(x_norm*y_norm)

In [18]:
# Test function
Cosine_Similarity(scores_predict[:,0], scores_predict[:,4])

0.628695176390365

In [19]:
def Adjusted_Cosine_Similarity(x,y):
    x_mean = np.mean(x)
    x_adjusted = x - x_mean
    
    y_mean = np.mean(y)
    y_adjusted = y - y_mean
    
    return Cosine_Similarity(x_adjusted, y_adjusted)

In [20]:
# Test function
Adjusted_Cosine_Similarity(scores_predict[:,0], scores_predict[:,4])

0.039389185138644874

In [21]:
def predict_scores_in_matrix(scores_predict, scores_actual, method_prediction, method_calculation, k):
    absolute_error = 0
    number_of_zeros = 0
    for idx, x in np.ndenumerate(scores_predict):
        if x == 0:
            predicted_score = predict_score_in_place(idx[0], idx[1], scores_predict, method_prediction, method_calculation, k)
            actual_score = scores_actual[idx[0],idx[1]]
            absolute_error += np.absolute(predicted_score - actual_score)
            number_of_zeros += 1
    return absolute_error/number_of_zeros            
            

In [22]:
predict_scores_in_matrix(scores_predict, method_prediction = 'Adjusted Cosine', method_calculation = 'Simple Average', k = 2)

TypeError: predict_scores_in_matrix() missing 1 required positional argument: 'scores_actual'

In [23]:
def predict_score_in_place(n_row, n_col, score_matrix, method_prediction, method_calculation, k):
    similarities = []
    for j in range(len(score_matrix[n_row])):
        current_column = j
        if j == n_col:
            continue
        else:
            vector_01 = []
            vector_02 = []
            for i in range(len(score_matrix[:,current_column])):
                if score_matrix[i,current_column] != 0 and score_matrix[i,n_col] != 0:
                    vector_01.append(score_matrix[i,current_column])
                    vector_02.append(score_matrix[i,n_col])
            # Make some changes
            similarity = choose_comparision_method(method_prediction, vector_01, vector_02)
            weight = Adjusted_Cosine_Similarity(vector_01, vector_02)
            similarities.append((similarity, current_column, weight))
    return calculate_score(score_matrix, similarities, n_row, k, method_calculation)

In [24]:
def choose_comparision_method(name_of_method, vector_01, vector_02):
    if name_of_method == 'Jaccard':
        return Jaccard_Similarity(vector_01, vector_02)
    elif name_of_method == 'Dice':
        return Dice_Similarity(vector_01, vector_02)
    elif name_of_method == 'Cosine':
        return Cosine_Similarity(vector_01, vector_02)
    elif name_of_method == 'Adjusted Cosine':
        return Adjusted_Cosine_Similarity(vector_01, vector_02)

In [25]:
def calculate_score(score_matrix, similarity_matrix, row, k, method):
    similarity_matrix.sort(key=lambda x:x[0], reverse=True)
    sum_scores = 0
    if method == 'Simple Average':
        for i in range(k):
            column = similarity_matrix[i][1]
            score = score_matrix[row][column]
            sum_scores += score
        return sum_scores/k
    elif method == 'Weighted Average':
        sum_weights = 0;
        for i in range(k):
            column = similarity_matrix[i][1]
            weight = similarity_matrix[i][2]
            score = score_matrix[row][column]
            weighted_score = weight*score
            sum_scores += weighted_score
            sum_weights += weight
        return sum_scores/sum_weights

In [26]:
def item_to_item_collaborative_filtering(n_rows, n_cols, percent, method_prediction, method_calculation, k, n_iters):
    scores_matrix = create_matrix(n_rows, n_cols)
    for i in range(n_iters):
        scores_to_predict = calculate_scores_for_prediction(scores_matrix, percent)
        mean_absolute_error = predict_scores_in_matrix(scores_to_predict, scores_matrix,method_prediction,method_calculation, k)
        print("For iteration ", i+1, " the mae is: ", mean_absolute_error)

In [34]:
item_to_item_collaborative_filtering(20,20,0.5,'Cosine','Simple Average',5,10)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
C:\Users\user\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\user\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


For iteration  1  the mae is:  3.5922527191317677
For iteration  2  the mae is:  3.3066024702528667
For iteration  3  the mae is:  3.6501776725785002
For iteration  4  the mae is:  3.3258706150956936
For iteration  5  the mae is:  3.2804548445173247
For iteration  6  the mae is:  3.5306203665967746
For iteration  7  the mae is:  3.3312837462296367
For iteration  8  the mae is:  3.508216971972398
For iteration  9  the mae is:  3.5240284325448847
For iteration  10  the mae is:  3.7109542607465915
